In [1]:
import pandas as pd
import re
# from sqlalchemy import create_engine

In [2]:
# Change file names as needed
# Input Files:
SOURCE_infile = './_sources/20190107_info_category_id_6_phase_2.csv'
TARGETS_infile = './_sources/dico_cible.csv'
LEVELS_infile = './_sources/dico_niveau.csv'
# Output Files
SENTENCES_outfile = './_output/sources.csv'
RESULTS_outfile = './_output/regles.csv'

In [3]:
# change as needed, but not the ordering
COLUMN_HEADERS = ['source_id',
 'niveau',
 'montant',
 'unite',
 'ordre',
 'order_niveau_regle',
 'cible',
 'flag_chiffre_independant_des_benefs',
                  'exception',
 'cible_si_independant',
 'flag_optionel',
    'min',
 'max',
 'flag_cible_multiple',
 'flag_delegation']

In [4]:
def readFile(filename):
    """reads given data"""
    with open(filename, encoding='utf-8') as f:
        lines = [s.strip('\n').strip('"').lower() for s in f.readlines()]
        # on commente la ligne suivane car on homogénise directement dans la source et les dicos
        # lines = [s.replace("'", "’") for s in lines] # for consistency
        for fr in ['fr.', 'frs']: # use francs consistently
            lines = [s.replace(fr, "francs") for s in lines] 
    return lines

In [5]:
def levelMarkers(s, levels):
    """replaces level key words with '|'"""
    for m in levels:
        s = re.sub(m, "|", s)
    return s

def splitIntoLevels(s, levels):
    """splits s into levels"""
    sl = levelMarkers(s, levels)
    return sl.split('|')

In [6]:
def splitOnPunctuations(s):
    """splits s on punctuations"""
    return [e for e in re.split('[?.,:;]', s) if len(e)>0]

In [7]:
def splitFurther(s):
    """partitions text with multiple % into list of strings with single %"""
    indices = [m.start(0) for m in re.finditer(r"(\d+ *\%)|(\d+\.\d+ *\%)|(\d+ *\d* */ *\d+ *\%)", s)]
    indices.append(None)
    return [s[indices[i]:indices[i+1]] for i in range(len(indices)-1)]

In [8]:
def getUnits(s, units):
    if re.search("%" , s):
        return "%"
    for u in units:
        if re.search(u , s):
            return "Francs"
    return None

In [9]:
def getValue(s):
    """returns value from text"""
    s = re.split('% |francs ',s)
    s = ''.join([re.sub('[^\d ./]', '', e) for e in s]).split()
    if len(s)>0:
        return s[0]
    return ''

In [10]:
def bestMatchBak(s, choices):
    """returns likeliest beneficiery from CHOICES"""
    m = [c for c in choices if c in s]
    if m: # if exact match
        mV = max([len(e) for e in m]) 
        return [x for x in m if len(x)==mV][0] # return longest string 
    return None

In [11]:
def bestMatch(s, choices):
    """returns likeliest beneficiery from CHOICES"""
    choices_lower = list(map(lambda choice: choice.lower(), choices)) # lower version of choices
    s_lower = s.lower() # lower of the string
    m = [c for c in choices_lower if c in s_lower]
    if m: # if exact match
        mV = max([len(e) for e in m])
        match = [x for x in m if len(x)==mV][0] # return longest string
        start = s_lower.find(match) # find the position of the lower version
        return s[start:start+len(match)] # return the same position for the raw version
    return None

In [12]:
def present(s, indicators):
    """returns True if indicator present in string s"""
    return any([f in s for f in indicators])

In [13]:
def setFlags(s, indicators):
    flags = []
    for ind in indicators:
        flags.append(present(s, ind))
    return flags

In [14]:
# Write SENTENCES_outfile
df = pd.read_csv(SOURCE_infile)
df.drop('SOURCE', axis=1, inplace=True)
df.to_csv(SENTENCES_outfile, index=False, header=['id', 'corporation', 'infodate', 'texte'])

In [15]:
# Read input files
EXAMPLES = list(df.COMMENTS)
TARGETS = readFile(TARGETS_infile)
LEVELS = readFile(LEVELS_infile)
UNITS = ['francs'] # not using 'dico_units.csv' since readFile takes care of it  

for n,v in zip(['EXAMPLES', 'TARGETS', 'LEVELS', 'UNITS'],[EXAMPLES, TARGETS, LEVELS, UNITS]):
    print('% s contains %d items' %(n, len(v)))

EXAMPLES contains 13712 items
TARGETS contains 207 items
LEVELS contains 21 items
UNITS contains 1 items


In [16]:
df.head()

,ID,CORPORATION,INFODATE,COMMENTS
0,304754,2100,31/12/1912,Répartition des bénéfices : effectuée suivant ...
1,304770,148,31/12/1912,Répartition des bénéfices : effectuée suivant ...
2,304789,436,31/12/1912,Répartition des bénéfices : effectuée suivant ...
3,304815,3246,31/12/1912,Répartition des bénéfices : 5 % d'intérêt aux ...
4,304830,3524,31/12/1912,Répartition des bénéfices : 20 % à la réserve ...


In [17]:
# Examples split into levels
EX_LEV = [] # Examples split into levels
for i,s in enumerate(EXAMPLES):
    EX_LEV.append(splitIntoLevels(s, LEVELS))

In [18]:
id_ = [] # Examples split into levels
for i,s in enumerate(EXAMPLES):
    id_.append(splitIntoLevels(s, LEVELS))

In [19]:
# Flag indicators
ind_ben = ["intérêt"]
ind_except = ["sauf"]
ind_optional = ["facultatif", "facultative", "facultatifs"]
ind_min = ['minimum']
ind_max = ['maximum', 'jusqu’à']
ind_target_multiple =['chaque', 'chacune', 'chacun']
ind_delegation = ['délégués', 'délégué', 'déléguée', 'déléguées']
indicators = [ind_ben, ind_except, ind_optional, ind_min,ind_max, ind_target_multiple, ind_delegation]

In [20]:
def minmax():
    if not flags[3]:
        flags[3] = False
    else:
        b = re.sub(r"(\d+ *\%)|(\d+\.\d+ *\%)|(\d+ *\d* */ *\d+ *\%)", "", s).strip()
        flags[3] = b
    if not flags[4]:
        flags[4] = False
    else:
        c = re.sub(r"(\d+ *\%)|(\d+\.\d+ *\%)|(\d+ *\d* */ *\d+ *\%)", "", s).strip()
        flags[4] = c
    return flags[3], flags[4]

In [21]:
%%capture
# Parse comments
res = []
#for i,eList in enumerate(df.id,1):
#for i,eList in zip(df.ID, EX_LEV):
#for m,eList in enumerate(EX_LEV,1) and i, _ in zip(df.ID, EX_LEV):
for i, eList in zip(df.ID, EX_LEV):   
    or_ct = 0 # order_rule
    for j,sL in enumerate(eList,1):
        olr_ct = 0 # order_level_rule
        for s1 in splitOnPunctuations(sL):
            if s1.count('%')> 1:
                sp = splitFurther(s1)
            else:
                sp = [s1]
            
            for s in sp:
                if len(s)>1:
                    if '%' in s or 'francs' in s or "sauf" in s or "maximum" in s or "minimum" in s:
                        flags = setFlags(s, indicators)
                        # flags
                        u = getUnits(s, UNITS)
                        v = getValue(s)
                        t = bestMatch(s, TARGETS)
                        if t: # match
                            or_ct += 1
                            olr_ct += 1
                            
                            if not flags[1]:
                                # exception indicator False
                                minmax()
                                flags = flags[:1]+[False]+flags[1:]
                                res.append([i,j,v,u,or_ct,olr_ct,t]+flags)
                            else:  # exception indicator True
                                minmax()
                                b = re.sub(r"(\d+ *\%)|(\d+\.\d+ *\%)|(\d+ *\d* */ *\d+ *\%)", "", s).strip()
                                flags = flags[:1]+[b]+flags[1:]
                                res[-1] = (res[-1][:7]+flags)

In [22]:
df.head(10)

,ID,CORPORATION,INFODATE,COMMENTS
0,304754,2100,31/12/1912,Répartition des bénéfices : effectuée suivant ...
1,304770,148,31/12/1912,Répartition des bénéfices : effectuée suivant ...
2,304789,436,31/12/1912,Répartition des bénéfices : effectuée suivant ...
3,304815,3246,31/12/1912,Répartition des bénéfices : 5 % d'intérêt aux ...
4,304830,3524,31/12/1912,Répartition des bénéfices : 20 % à la réserve ...
5,304847,2346,31/12/1912,Répartition des bénéfices : 20% à la réserve l...
6,304866,2120,31/12/1912,Répartition des bénéfices : 20 % à la réserve ...
7,304881,365,31/12/1912,Répartition des bénéfices : 20 % à la réserve ...
8,304898,2115,31/12/1912,Répartition des bénéfices : 20 % à la réserve ...
9,304922,1594,31/12/1912,Répartition des bénéfices : après déduction de...


In [23]:
# Write results into outputFile
df1 = pd.DataFrame(res, columns=COLUMN_HEADERS)
df1.to_csv(RESULTS_outfile, index=False)

In [24]:
df1.head(10)

,source_id,niveau,montant,unite,ordre,order_niveau_regle,cible,flag_chiffre_independant_des_benefs,exception,cible_si_independant,flag_optionel,min,max,flag_cible_multiple,flag_delegation
0,304815,1,5,%,1,1,intérêt aux actions,True,False,False,False,False,False,False,False
1,304815,2,10,%,2,1,réserve,False,False,False,False,False,False,False,False
2,304815,2,5,%,3,2,Conseil d'administration,False,False,False,False,False,False,False,False
3,304815,2,3,%,4,3,Directeur,False,False,False,False,False,False,False,False
4,304815,2,2,%,5,4,Caisse de retraite des employés,False,False,False,False,False,False,False,False
5,304815,3,20,%,6,1,parts de fondateur,False,False,False,False,False,False,False,False
6,304815,3,10,%,7,2,réserve spéciale pour amortissements,False,False,False,False,False,False,False,False
7,304830,1,20,%,1,1,réserve légale,False,False,False,False,False,False,False,False
8,304847,1,20,%,1,1,réserve légale,False,False,False,False,False,False,False,False
9,304847,1,5,%,2,2,intérêt aux actions,True,False,False,False,False,False,False,False


In [25]:
# disk_engine = create_engine("sqlite:///dfih_rules")

In [26]:
# df.to_sql("sources", disk_engine, if_exists = "replace")
# df1.to_sql("rules", disk_engine, if_exists = "replace")

In [27]:
LEVELS = readFile(LEVELS_infile)

In [28]:
LEVELS

['le surplus',
 'sur le surplus',
 'après affectation',
 'ensuite',
 'le solde',
 'sur le solde',
 "l'excédent",
 "sur l'excédent",
 'sur le reliquat',
 'après amortissement de toutes les actions',
 'après ces divers prélèvements',
 'le reste des bénéfices',
 'après paiement',
 'après la répartition',
 'sur le reste',
 'du surplus',
 'le reliquat',
 'le solde disponible',
 'sur le disponible',
 'sur le restant',
 'sur le nouveau solde subsistant']